In [1]:
%load_ext autoreload

In [2]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

if not "../data_processing" in sys.path:
    sys.path.append("../data_processing")

%autoreload
import build_dataset

ROOT /Users/mortenolsenosvik/Documents/NTNU/Master/master-thesis/src
appended
/Users/mortenolsenosvik/Documents/NTNU/Master/master-thesis/src


In [43]:
path = '/Volumes/OsvikExtra/VibrationData/RMS_dataset/GbxHssRr_RMS_power>WTG02.csv'
data = pd.read_csv(path)
data.head()

,AvgPower,ActPower,AvgRotSpeed,WindSpeed,NacelleDirection,GbxHssRr_RMS_0,GbxHssRr_RMS_1,GbxHssRr_RMS_2,GbxHssRr_RMS_3,GbxHssRr_RMS_4,...,GbxHssRr_RMS_40,GbxHssRr_RMS_41,GbxHssRr_RMS_42,GbxHssRr_RMS_43,GbxHssRr_RMS_44,GbxHssRr_RMS_45,GbxHssRr_RMS_46,GbxHssRr_RMS_47,GbxHssRr_RMS_48,GbxHssRr_RMS_49
0,406.781433,431.200012,966.748980,5.2,208.300003,0.183803,0.578023,0.382599,0.713316,0.137359,...,0.133423,0.223895,0.186688,0.218882,0.279001,0.271950,0.138354,0.089420,0.073151,0.044012
1,2818.207520,3291.300049,1487.871218,14.2,226.699997,0.404392,0.258909,0.811581,0.344739,1.497359,...,1.101426,1.969857,1.778854,2.673142,3.386242,2.750154,1.419267,0.809299,0.444020,0.272188
2,2528.062988,2454.500000,1471.267571,10.6,231.699997,0.532580,0.244108,1.221325,0.346431,1.477624,...,1.106000,2.131452,1.824046,2.082840,2.086788,1.563331,0.788437,0.436628,0.294574,0.181487
3,2500.473145,1772.500000,1432.996560,8.4,284.600006,0.478452,0.210531,1.090423,0.360744,0.946055,...,0.967446,1.962500,1.463750,1.369159,1.247210,1.092017,0.519553,0.320830,0.221710,0.146560
4,467.837036,441.200012,1041.878054,6.2,322.100006,0.202902,2.325503,0.570255,0.546559,0.159730,...,0.149326,0.259508,0.219284,0.240134,0.273170,0.261065,0.138248,0.104824,0.079551,0.045207


In [44]:
data = data.drop(columns=['ActPower'])
def train_test_split(df, percentage):
    split_index = int(np.floor(df.shape[0]) * percentage)
    train = df[:split_index]
    test = df[split_index:].reset_index(drop=True)
    return train, test

train, test = train_test_split(data, 0.8)

In [45]:
print(train.shape)
print(test.shape)

(335, 54)
(84, 54)


## Loss function MAE (Mean Absolute Error)

In [46]:
def custom_mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true))

## Scale with StandardScaler

In [47]:
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

## PCA 

In [48]:
'''
The number of components required to capture 90% of the variance 
will be used
'''
pca = PCA(0.99) 
pca.fit(train_scaled)
print(f'The number of components is: {pca.n_components_}')

The number of components is: 21


In [49]:
train_pca = pca.transform(train_scaled)
train_pca_reconstructed = pca.inverse_transform(train_pca)
train_reconstruction_variance = np.var(train_scaled - pca.inverse_transform(train_pca))
print(f'The training reonstruction variance is: {train_reconstruction_variance}')

The training reonstruction variance is: 0.009313422566676844


In [50]:
test_pca = pca.transform(test_scaled)
test_pca_reconstructed = pca.inverse_transform(test_pca)
test_reconstruction_variance = np.var(test_scaled - pca.inverse_transform(test_pca))
print(f'The testing reonstruction variance is: {test_reconstruction_variance}')

The testing reonstruction variance is: 0.01452632052559108


In [51]:
train_loss = custom_mae(train_scaled, train_pca_reconstructed)
print(f'MAE loss on train data using pca: {train_loss}')

MAE loss on train data using pca: 0.06829479283843377


In [52]:
test_loss = custom_mae(test_scaled, test_pca_reconstructed)
print(f'MAE loss on test data using pca: {test_loss}')

MAE loss on test data using pca: 0.0871594859820424
